In [150]:
import pandas as pd
import numpy as np
import zipfile
from veryfi import Client
import textdistance
import re
import pickle

In [24]:
#import zipfile
#with zipfile.ZipFile("documents_to_process.zip","r") as zip_ref:
#    zip_ref.extractall("unzippedDocs")

In [4]:
#list_files = ['0001431487.jpg', '0060027768.jpg','0060027778.jpg','0060068160.jpg']


zip_file_path = 'documents_to_process.zip'

client_id = ''
client_secret = ''
username = ''
api_key = ''

veryfi_client = Client(client_id, client_secret, username, api_key)
response = veryfi_client.process_document(zip_file_path)

In [25]:
#responce2=[]
#for i in list_files:
#    responce2.append(veryfi_client.process_document('unzippedDocs/'+i))

In [17]:
docs=response['ocr_text'].split('\x0c')

In [40]:
counter=0
for i in docs:
    t=i.split('\n')
    t=[j.split('\t') for j in t]
    docs[counter]=t
    counter+=1

In [151]:
#with open('processedDocs.pkl','wb') as file:
#    pickle.dump(docs,file)

In [153]:
with open('processedDocs.pkl', 'rb') as f:
    docs = pickle.load(f)

In [117]:
#sup1: Kw1 in invoice
#sup2: name in caps and without numbers

def findKwLoc(kw,doc,sensibility=3):
    pi=0
    for i in doc:

        pj=0
        for j in i:
            dst=textdistance.damerau_levenshtein.distance(kw,j)
            
            if dst<sensibility:
                location=(pi,pj)
                return(location)
            pj+=1
        pi+=1
    

In [118]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

def has_lowercase(inputString):
    return bool(re.search(r'[a-z]', inputString))


In [119]:
kw1='BILLING INSTRUCTIONS MAL YOUR INVOICES IN DUPLICATE TO CONSIGNEE'

In [120]:
vendor_names=[]
for i in docs:
    poss=findKwLoc(kw1,i,3)
    if poss==None:
        vendor_names.append(-1)
        continue
    options=[j[0] for j in i[0:poss[0]]]
    boli=[not (has_numbers(j) or has_lowercase(j) or j=='') for j in options]
    options=list(np.array(options)[boli])
    vendor_names.append(' '.join(options))
    
    
    

In [121]:
vendor_names

['THE AMERICAN TOBACCO COMPANY',
 'THE AMERICAN TOBACCO COMPANY',
 'THE AMERICAN TOBACCO COMPANY',
 'THE AMERICAN TOBACCO COMPANY']

In [123]:
kw2='AS SHOWN BELOW'
kw3='SHIPMENT TO ARRIVE NOT LATER THAN'

In [138]:
#sup3: Kw2 in invoice
#sup4: Kw3 in invoice
#sup5: "TO:" not in shipping details 
#sup6: shipping details has more than 3 lines

shippinngDeets=[]
for i in docs:
    poss1=list(findKwLoc(kw2,i,3))
    poss2=list(findKwLoc(kw3,i,3))
    if poss1==None or poss2==None:
        shippinngDeets.append(-1)
        continue
    poss1[0]=poss1[0]+2
    poss2[0]=poss2[0]-1
    
    shippinngDeets.append(i[poss1[0]:poss2[0]])

In [146]:
# use TO: to further delimit the search
for i in range(len(shippinngDeets)):
    frst3=shippinngDeets[i][0:3]
    frst3=[' '.join(j) for j in frst3]
    counter=0
    index=-1
    for j in frst3:
        if 'TO:' in j:
            index=counter
        counter+=1
    if index==-1:
        continue
    [shippinngDeets[i].pop(0) for k in range(index+1)]

In [147]:
shippinngDeets

[[['EDEN JEWELRY CO., INC.', '', 'H THE FULFILLMENT HOUSE'],
  ['212 Highlawn Avenue', '', '', '7 Midland Avenue'],
  ['Brooklyn, NY 11223', '', 'P', 'Hicksville, NY', '11801'],
  ['ATTN: Mr. Dennis Flyer.', '', 'ATTN: Mr. Bob Pollack']],
 [['U', 'MACNAUGHTON EINSON GRAPHICS', 'AS PER ATTACHED INSTRUCTIONS'],
  ['Twenty-Ten Maple Avenue', '', 'Balance To Be Held In Inventory'],
  ['P'],
  ['Fairlawn, NJ 07410', '', 'P'],
  ['ATTN: Mr. Denis Sullivan'],
  ['', 'T'],
  ['', '0']],
 [['U', 'ARISTON,', 'INCORPORATED', '', '$', 'AS PER ATTACHED INSTRUCTIONS'],
  ['485 Bloy Street', '', '', 'Balance To Be Held In Inventory'],
  ['P', 'Hillside, NJ', '07205'],
  ['L'],
  ['ATTN: Mr. Warren Katz'],
  ['T'],
  ['E']],
 [['Kubin-icholson', '', '', 'Per Shipping Instructions,'],
  ['Atta: Mr. 2. L. Gillon', '', 'Attached'],
  ['60 East 42nd Street'],
  ['New York, NY 10017'],
  ['1']]]